In [73]:
# from requests import get
import requests
import re
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
import random
import numpy as np, pandas as pd
import seaborn as sns
import urllib.request
from tqdm import tqdm

In [97]:
# Test
url = "https://en.wikipedia.org/wiki/Asomiya_Khabar"
response = requests.get(url=url)

soup = BeautifulSoup(response.content, 'html.parser')
des_num = len(soup.find_all("p"))
print(des_num)
description_unfiltered = str(soup.find_all("p")[0])
description = re.sub(r"<.*?>", "", description_unfiltered)
print(description_unfiltered)
print(description)
if description == str("\n"):
    description_unfiltered = str(soup.find_all("p")[1])
    description = re.sub(r"<.*?>", "", description_unfiltered)
    print(description_unfiltered)
    print(description)
    
allLinks = soup.find(id="bodyContent").find_all("a")
print(allLinks[0]["href"])

3
<p class="mw-empty-elt">
</p>


<p><b><span title="Assamese-language text"><i lang="as-Latn">Asomiya Khabar</i></span></b> (<a href="/wiki/Assamese_language" title="Assamese language">Assamese:</a> অসমীয়া খবৰ) (or <i><b>Oxomiya Khobor</b></i>) is an <a href="/wiki/Assamese_language" title="Assamese language">Assamese</a> daily newspaper published simultaneously from <a href="/wiki/Guwahati" title="Guwahati">Guwahati</a> and <a href="/wiki/Jorhat" title="Jorhat">Jorhat</a>.<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> It is one of the highest circulated Assamese daily newspaper. A weekly supplement named <i>Deoboriya Khabar</i> is published with it on every Sunday. It was placed as the second largest vernacular daily of the region.<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup> Veteran journalist Shankar Laskar is in the position of editor of <span title="Assamese-language text"><i lang="as-Latn">Asomiya Khabar</i></span> and 

In [111]:
Dont_include_in_links = r"User|File\:|wiki\/Wikipedia\:|Special\:|Help\:|\#|Template|Category\:"

def Scraper(url, data_frame):
    try:
        response = requests.get(url=url)
    except:
        sleep(5)
        print("Sleeping...")
        response = requests.get(url=url)

    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find(id="firstHeading")
    try:
        description_unfiltered = str(soup.find_all("p")[0])
        # Make description readable
        description = re.sub(r"<.*?>", "", description_unfiltered)
    except:
        print("Cannot find paragraph?")
        description = title.text
    
    if description == str("\n"):
        try:
            description_unfiltered = str(soup.find_all("p")[1])
            description = re.sub(r"<.*?>", "", description_unfiltered)
        except:
            print("Cannot find paragraph??")
            description = title.text
            
    df_temp = pd.DataFrame([[title.text, description, url]], columns=["Title", "Description", "Link"])
    data_result = pd.concat([data_frame, df_temp], ignore_index=True)

    # Randomly get a next link
    allLinks = soup.find(id="bodyContent").find_all("a")
    random.shuffle(allLinks)
    linkToScrape = 0
    
    for link in allLinks:
        # Only interested in other wiki articles
        try:
            if link['href'].find("/wiki/") == -1 or \
                len(re.findall(Dont_include_in_links, link['href'])) >= 1: #or link['href'].find("\."): 
                continue
        except:
            print("Error! Check next link...")
            continue

        # Use this link to scrape
        linkToScrape = link
        break

    # The next link to scrape
    next_link = "https://en.wikipedia.org" + linkToScrape['href']
    # The link should be valid
    next_link = re.sub(r"^https.*https", "https", next_link)
    # print(title.text)
    # print(next_link)
    
    return data_result, next_link

In [14]:
df = pd.read_csv("data.csv", index_col=False)
next_link = "https://en.wikipedia.org/wiki/Hans_B%C3%B8rli"
display(df)

,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
3,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
4,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
...,...,...,...
284,The Hurricane Party,The Hurricane Party (Swedish: Orkanpartyt) is ...,https://en.wikipedia.org/wiki/The_Hurricane_Party
285,Category:Albert Bonniers Förlag books,"The following 55 pages are in this category, o...",https://en.wikipedia.org/wiki/Category:Albert_...
286,The Sorrow Gondola,The Sorrow Gondola (Swedish: Sorgegondolen) is...,https://en.wikipedia.org/wiki/The_Sorrow_Gondola
287,Tomas Tranströmer,Tomas Gösta Tranströmer (.mw-parser-output .IP...,https://en.wikipedia.org/wiki/Tomas_Transtr%C3...


In [131]:
next_link = "https://en.wikipedia.org/wiki/Google_Checkout"

In [132]:
for i in tqdm(range(0, 1000)):
    df, next_link = Scraper(next_link, df)
    
display(df)
print(next_link)

  5%|████                                                                            | 51/1000 [00:23<04:50,  3.27it/s]

Error! Check next link...


 18%|██████████████                                                                 | 178/1000 [01:21<07:50,  1.75it/s]

Error! Check next link...


 19%|██████████████▊                                                                | 188/1000 [01:27<10:28,  1.29it/s]

...

 96%|███████████████████████████████████████████████████████████████████████████▌   | 957/1000 [07:08<00:18,  2.35it/s]

Error! Check next link...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [07:28<00:00,  2.23it/s]


,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
9946,Creative Commons license,A Creative Commons (CC) license is one of seve...,https://en.wikipedia.org/wiki/Creative_Commons...
9947,Portal:Free and open-source software,Free and open-source software (FOSS) is softwa...,https://en.wikipedia.org/wiki/Portal:Free_and_...
9948,Tiresias (typeface),Tiresias is a family of TrueType sans-serif ty...,https://en.wikipedia.org/wiki/Tiresias_(typeface)
9949,MetaType1,"MetaType1, also stylized as METATYPE1, is a to...",https://en.wikipedia.org/wiki/MetaType1


https://en.wikipedia.org/wiki/Ubuntu_Forums


In [135]:
df = df.drop_duplicates(subset=["Title"])
display(df)
print(next_link)

,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
9944,Stack (abstract data type),"In computer science, a stack is an abstract da...",https://en.wikipedia.org/wiki/Stack_(data_stru...
9946,Creative Commons license,A Creative Commons (CC) license is one of seve...,https://en.wikipedia.org/wiki/Creative_Commons...
9948,Tiresias (typeface),Tiresias is a family of TrueType sans-serif ty...,https://en.wikipedia.org/wiki/Tiresias_(typeface)
9949,MetaType1,"MetaType1, also stylized as METATYPE1, is a to...",https://en.wikipedia.org/wiki/MetaType1


https://en.wikipedia.org/wiki/Ubuntu_Forums


In [136]:
df.to_csv('data.csv', index=False)

In [137]:
read_df = pd.read_csv("data.csv", index_col=False)
display(read_df)

,Title,Description,Link
0,Web scraping,"Web scraping, web harvesting, or web data extr...",https://en.wikipedia.org/wiki/Web_scraping
1,Wrapper (data mining),Wrapper in data mining is a procedure that ext...,https://en.wikipedia.org/wiki/Wrapper_(data_mi...
2,Microformat,Microformats (μF)[note 1] are a set of defined...,https://en.wikipedia.org/wiki/Microformat
3,Common Logic,Common Logic (CL) is a framework for a family ...,https://en.wikipedia.org/wiki/Common_Logic
4,ISLISP,ISLISP (also capitalized as ISLisp) is a progr...,https://en.wikipedia.org/wiki/ISLISP
...,...,...,...
9158,Stack (abstract data type),"In computer science, a stack is an abstract da...",https://en.wikipedia.org/wiki/Stack_(data_stru...
9159,Creative Commons license,A Creative Commons (CC) license is one of seve...,https://en.wikipedia.org/wiki/Creative_Commons...
9160,Tiresias (typeface),Tiresias is a family of TrueType sans-serif ty...,https://en.wikipedia.org/wiki/Tiresias_(typeface)
9161,MetaType1,"MetaType1, also stylized as METATYPE1, is a to...",https://en.wikipedia.org/wiki/MetaType1
